## Load in data

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/datadrivendecisions/duckdb-demo/refs/heads/main/ev_charging_patterns.csv"
df = pd.read_csv(url)
df.head()

,User ID,Vehicle Model,Battery Capacity (kWh),Charging Station ID,Charging Station Location,Charging Start Time,Charging End Time,Energy Consumed (kWh),Charging Duration (hours),Charging Rate (kW),Charging Cost (USD),Time of Day,Day of Week,State of Charge (Start %),State of Charge (End %),Distance Driven (since last charge) (km),Temperature (°C),Vehicle Age (years),Charger Type,User Type
0,User_1,BMW i3,108.463007,Station_391,Houston,2024-01-01 00:00:00,2024-01-01 00:39:00,60.712346,0.591363,36.389181,13.087717,Evening,Tuesday,29.371576,86.119962,293.602111,27.947953,2.0,DC Fast Charger,Commuter
1,User_2,Hyundai Kona,100.000000,Station_428,San Francisco,2024-01-01 01:00:00,2024-01-01 03:01:00,12.339275,3.133652,30.677735,21.128448,Morning,Monday,10.115778,84.664344,112.112804,14.311026,3.0,Level 1,Casual Driver
2,User_3,Chevy Bolt,75.000000,Station_181,San Francisco,2024-01-01 02:00:00,2024-01-01 04:48:00,19.128876,2.452653,27.513593,35.667270,Morning,Thursday,6.854604,69.917615,71.799253,21.002002,2.0,Level 2,Commuter
3,User_4,Hyundai Kona,50.000000,Station_327,Houston,2024-01-01 03:00:00,2024-01-01 06:42:00,79.457824,1.266431,32.882870,13.036239,Evening,Saturday,83.120003,99.624328,199.577785,38.316313,1.0,Level 1,Long-Distance Traveler
4,User_5,Hyundai Kona,50.000000,Station_108,Los Angeles,2024-01-01 04:00:00,2024-01-01 05:46:00,19.629104,2.019765,10.215712,10.161471,Morning,Saturday,54.258950,63.743786,203.661847,-7.834199,1.0,Level 1,Long-Distance Traveler


## Query with DuckDB

In [2]:
import duckdb

duckdb.sql(f"SELECT * from '{url}'")

┌───────────┬───────────────┬────────────────────────┬─────────────────────┬───────────────────────────┬─────────────────────┬─────────────────────┬───────────────────────┬───────────────────────────┬────────────────────┬─────────────────────┬─────────────┬─────────────┬───────────────────────────┬─────────────────────────┬──────────────────────────────────────────┬─────────────────────┬─────────────────────┬─────────────────┬────────────────────────┐
│  User ID  │ Vehicle Model │ Battery Capacity (kWh) │ Charging Station ID │ Charging Station Location │ Charging Start Time │  Charging End Time  │ Energy Consumed (kWh) │ Charging Duration (hours) │ Charging Rate (kW) │ Charging Cost (USD) │ Time of Day │ Day of Week │ State of Charge (Start %) │ State of Charge (End %) │ Distance Driven (since last charge) (km) │  Temperature (°C)   │ Vehicle Age (years) │  Charger Type   │       User Type        │
│  varchar  │    varchar    │         double         │       varchar       │          va

## Create a database and write

In [3]:
# create a connection to a file called 'file.db'
con = duckdb.connect("file.db")
# drop the table
con.sql("DROP TABLE IF EXISTS test")
# create a table and load data into it
con.sql("CREATE TABLE test (i FLOAT, text VARCHAR)")
con.sql("INSERT INTO test VALUES (42, 'Hello')")
# query the table
con.table("test").show()
# explicitly close the connection
con.close()
# Note: connections also closed implicitly when they go out of scope

┌───────┬─────────┐
│   i   │  text   │
│ float │ varchar │
├───────┼─────────┤
│  42.0 │ Hello   │
└───────┴─────────┘



## Create table from csv

In [4]:
# create query string
query = f"""
        CREATE TABLE test2 AS
            SELECT *
            FROM read_csv('{url}');
        """
# create a connection to a file called 'file.db'
con = duckdb.connect("file.db")
# drop the table
con.sql("DROP TABLE IF EXISTS test2")
# create table from csv
con.sql(query)
# query the table
con.table("test2").show()
# explicitly close the connection
con.close()

┌───────────┬───────────────┬────────────────────────┬─────────────────────┬───────────────────────────┬─────────────────────┬─────────────────────┬───────────────────────┬───────────────────────────┬────────────────────┬─────────────────────┬─────────────┬─────────────┬───────────────────────────┬─────────────────────────┬──────────────────────────────────────────┬─────────────────────┬─────────────────────┬─────────────────┬────────────────────────┐
│  User ID  │ Vehicle Model │ Battery Capacity (kWh) │ Charging Station ID │ Charging Station Location │ Charging Start Time │  Charging End Time  │ Energy Consumed (kWh) │ Charging Duration (hours) │ Charging Rate (kW) │ Charging Cost (USD) │ Time of Day │ Day of Week │ State of Charge (Start %) │ State of Charge (End %) │ Distance Driven (since last charge) (km) │  Temperature (°C)   │ Vehicle Age (years) │  Charger Type   │       User Type        │
│  varchar  │    varchar    │         double         │       varchar       │          va

## SELECT statement

In [8]:

# create a connection to a file called 'file.db'
con = duckdb.connect("file.db")
# create query string
query = f"""
            DESCRIBE test2;
        """
# run query
con.execute(query)
print(con.fetchall())
# explicitly close the connection
con.close()

[('User ID', 'VARCHAR', 'YES', None, None, None), ('Vehicle Model', 'VARCHAR', 'YES', None, None, None), ('Battery Capacity (kWh)', 'DOUBLE', 'YES', None, None, None), ('Charging Station ID', 'VARCHAR', 'YES', None, None, None), ('Charging Station Location', 'VARCHAR', 'YES', None, None, None), ('Charging Start Time', 'TIMESTAMP', 'YES', None, None, None), ('Charging End Time', 'TIMESTAMP', 'YES', None, None, None), ('Energy Consumed (kWh)', 'DOUBLE', 'YES', None, None, None), ('Charging Duration (hours)', 'DOUBLE', 'YES', None, None, None), ('Charging Rate (kW)', 'DOUBLE', 'YES', None, None, None), ('Charging Cost (USD)', 'DOUBLE', 'YES', None, None, None), ('Time of Day', 'VARCHAR', 'YES', None, None, None), ('Day of Week', 'VARCHAR', 'YES', None, None, None), ('State of Charge (Start %)', 'DOUBLE', 'YES', None, None, None), ('State of Charge (End %)', 'DOUBLE', 'YES', None, None, None), ('Distance Driven (since last charge) (km)', 'DOUBLE', 'YES', None, None, None), ('Temperature (°

In [12]:
# create a connection to a file called 'file.db'
con = duckdb.connect("file.db")
# create query string
query = f"""
            SELECT *
            FROM test2
            WHERE "Vehicle Model" ILIKE '%BMW%';
        """
# run query
con.execute(query)
print(con.fetchall())
# explicitly close the connection
con.close()

[('User_1', 'BMW i3', 108.46300741284072, 'Station_391', 'Houston', datetime.datetime(2024, 1, 1, 0, 0), datetime.datetime(2024, 1, 1, 0, 39), 60.71234573492677, 0.5913634253585007, 36.3891805669814, 13.087716791774506, 'Evening', 'Tuesday', 29.371575979140047, 86.11996244457839, 293.6021106383279, 27.947953055800944, 2.0, 'DC Fast Charger', 'Commuter'), ('User_11', 'BMW i3', 50.0, 'Station_206', 'Los Angeles', datetime.datetime(2024, 1, 1, 10, 0), datetime.datetime(2024, 1, 1, 13, 59), 18.206197477378716, 2.6486354477276968, 10.310701112892176, 38.221230704051806, 'Night', 'Thursday', 76.08656557040186, 82.2655816267944, 64.63464374887128, 0.5004335643954789, 3.0, 'DC Fast Charger', 'Casual Driver'), ('User_15', 'BMW i3', 85.0, 'Station_96', 'Houston', datetime.datetime(2024, 1, 1, 14, 0), datetime.datetime(2024, 1, 1, 17, 32), 12.927545290947553, 3.4218093378407497, 24.211150117119175, 18.552815677456245, 'Afternoon', 'Monday', 71.68354383511853, 63.70663794066834, 155.36060543681995